In [17]:
import pandas as pd
import json
import re
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
import stanza

In [22]:
import stanza

# Download and initialize the English model
stanza.download('en')  # This line is valid if you're using an older version. If not, skip it.
nlp = stanza.Pipeline('en'

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1020)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1020)>


AttributeError: module 'stanza' has no attribute 'download'

In [2]:
file_path = '../data/edos_labelled_individual_annotations.csv'  
df = pd.read_csv(file_path)
df

,rewire_id,text,annotator,label_sexist,label_category,label_vector,split
0,sexism2022_english-0,[USER] I wonder what keeps that witch looking ...,17,sexist,2. derogation,2.2 aggressive and emotive attacks,train
1,sexism2022_english-0,[USER] I wonder what keeps that witch looking ...,2,sexist,2. derogation,2.2 aggressive and emotive attacks,train
2,sexism2022_english-0,[USER] I wonder what keeps that witch looking ...,6,not sexist,none,none,train
3,sexism2022_english-1,"What do you guys think about female ""incels""? ...",17,not sexist,none,none,train
4,sexism2022_english-1,"What do you guys think about female ""incels""? ...",15,not sexist,none,none,train
...,...,...,...,...,...,...,...
59995,sexism2022_english-9998,"Since 1973, America has sanctioned the murder ...",2,sexist,4. prejudiced discussions,4.2 supporting systemic discrimination against...,test
59996,sexism2022_english-9998,"Since 1973, America has sanctioned the murder ...",16,not sexist,none,none,test
59997,sexism2022_english-9999,Laura Bates Lol she looks like she'd suck the ...,2,sexist,2. derogation,2.2 aggressive and emotive attacks,train
59998,sexism2022_english-9999,Laura Bates Lol she looks like she'd suck the ...,16,not sexist,none,none,train


In [3]:
df.rename(columns={'label_sexist': 'label'}, inplace=True)
df['label'] = df['label'].map({'sexist': 1, 'not sexist': 0})

In [16]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1020)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1020)>


False

In [19]:
nlp = stanza.Pipeline('en', processors='tokenize,lemma,pos')

AttributeError: module 'stanza' has no attribute 'Pipeline'